# Proyecto Álgebra Conmutativa: Calculando certificados de no viabilidad de problemas combinatorios a través del Nullstellensatz de Hilbert

#### Integrantes: Federico Gálvez Zuleta, Francisco Javier Díaz Perdomo  y Juan Camilo González Cabrera

In [20]:
using Oscar

In [21]:
function text_reader(name)
    f = open(name)
    lines = readlines(f)
    E = []
    v= 0
    for l in lines
        x = l[1]
        if x == 'p'
            t = split(l)
            v = parse(Int64, t[3])
        elseif x == 'e'
            t = split(l)
            push!(E, [parse(Int64,t[2]), parse(Int64,t[3])]) 
        end
    end
return v, E
end

function polisGrafo(g,k,R)
    # Devuelve dado un grafo y un entero k, un sistema de polinomios que ayuda a determinar que el grafo es k-colorable Y el anillo de polinomios
    ps = [R[j]^k-1 for j in 1:nv] # los polinomios x^k_i- 1 = 0, ∀i ∈ V(g)
    ps2 = [R[src(r)]^(k-1-l) * R[dst(r)]^(l) for r in collect(edges(g)), l in 0:k-1] # los polinomios sum_l=0^{k-1} x_i^{k-1-l} x_j^l=0 ∀{i, j} ∈ E(G)
    p2 = sum(ps2,dims=2)
    polis = vec(vcat(ps,p2)) # crea una lista con los polinomios
    return polis # retorna la lista de polinomios 
end
        

polisGrafo (generic function with 1 method)

In [22]:
nv, E = text_reader("Grafos/johnson8-2-4.clq.txt")
g = Graph{Undirected}(nv)
for v in E
    add_edge!(g, v[1], v[2])
end

R, x = PolynomialRing(QQ, ["x$i" for i in 1:nv]) # Definimos el anillo de polinomios con n = #vercies del grafo variables

(Multivariate Polynomial Ring in 28 variables x1, x2, x3, x4, ..., x28 over Rational Field, fmpq_mpoly[x1, x2, x3, x4, x5, x6, x7, x8, x9, x10  …  x19, x20, x21, x22, x23, x24, x25, x26, x27, x28])

In [23]:
G = polisGrafo(g, 3, R)
println(length(G))

238


In [24]:
#  Calcula las particiones de d en sumas de n numeros, pueden ser 0
function partition(n::Int, d::Int)
    if n == 1
        return [[d]]
    end
    partitions = []
    for i in 0:d
        for p in partition(n-1, d-i)
            push!(partitions, [i; p])
        end
    end
    return partitions
end
# Calcula los polinomios beta_i sin coeficientes
function polysBi(n::Int, d::Int,genes)
    v = []
    
    for j in 0:d
        v = vcat(v, [reduce(*,[genes[i]^par[i] for i in 1:length(par)]) for par in partition(n,j)])
    end
    return v
end
# Retorna los polinomios beta_i * f_i sin los coeficientes
function creaCertv(d,listap)
    v=[]
    n=length(gens(parent(listap[1])))
    for f in listap
        dd = total_degree(f)
        v = vcat(v,[ f * bi for bi in polysBi(n, d - dd,gens(parent(listap[1])))])
    end
    return v
end

creaCertv (generic function with 1 method)

In [25]:
println(nv+length(G))

266


In [26]:
d = maximum([total_degree(poly) for poly in G])
bifi = creaCertv(d,G)

6118-element Vector{Any}:
 x1^3 - 1
 x2^3 - 1
 x3^3 - 1
 x4^3 - 1
 x5^3 - 1
 x6^3 - 1
 x7^3 - 1
 x8^3 - 1
 x9^3 - 1
 x10^3 - 1
 ⋮
 x9*x15^2 + x9*x15*x28 + x9*x28^2
 x8*x15^2 + x8*x15*x28 + x8*x28^2
 x7*x15^2 + x7*x15*x28 + x7*x28^2
 x6*x15^2 + x6*x15*x28 + x6*x28^2
 x5*x15^2 + x5*x15*x28 + x5*x28^2
 x4*x15^2 + x4*x15*x28 + x4*x28^2
 x3*x15^2 + x3*x15*x28 + x3*x28^2
 x2*x15^2 + x2*x15*x28 + x2*x28^2
 x1*x15^2 + x1*x15*x28 + x1*x28^2

In [34]:
function nulla(polys,K)
    n = length(gens(parent(polys[1])))
    d = maximum([total_degree(poly) for poly in polys])
    while d <= K
        # Creacion del polinomio de  setificado CERT
        CERTsinC = creaCertv(d,polys)
        nUnkows = length(CERTsinC)
        T, y =  PolynomialRing(QQ,["y$i" for i in 1:nUnkows])
        R2, z = PolynomialRing(T,["z$i" for i in 1:n])
        PG = polisGrafo(g, 3, R2)
        CERTsinC2 = creaCertv(d,PG)
        CERTv = [ y[i]*CERTsinC2[i] for i in  1:length(CERTsinC2)]
        CERT = reduce(+,CERTv)
        CERTt = collect(terms(CERT))

        # Creacion del sistema lineal 
        A = []
        for cee in CERTt
            achiquita = vec([Int( numerator( coeff(cc, yi))) for yi in gens(T), cc in collect(coeffs(cee))])
            if isempty(A)
                A = achiquita
            else
                A=hcat(A,achiquita)
            end
        end
        A = transpose( A)

        b = zeros(Int,length(CERTt))
        b[end]+=1
        sol = A \ b
        if any(sol .!= 0) # si no es todo 0, el sistema es consistente
            println("The system of equations F is infeasible")
            return sol
        end
        println("Una iteracion")
        d += 1
    end
    println("The system of equations F is feasible.")
    return true
end

nulla (generic function with 1 method)

In [35]:
nulla(G,56)

The system of equations F is infeasible


11-element Vector{Float64}:
 -0.2833333333333332
 -0.2611111111111111
 -0.26111111111111096
 -3.110349676662262e-18
 -0.02777777777777775
  0.06666666666666658
  0.04444444444444461
  9.331049029986785e-18
  0.06666666666666667
 -0.027777777777777842
  0.04444444444444448

In [30]:
listap =[x[1]^3 - 1
,x[2]^3 - 1
,x[3]^3 - 1
,x[1]^2 + x[1]*x[2] + x[2]^2
,x[1]^2 + x[1]*x[3] + x[3]^2]

5-element Vector{fmpq_mpoly}:
 x1^3 - 1
 x2^3 - 1
 x3^3 - 1
 x1^2 + x1*x2 + x2^2
 x1^2 + x1*x3 + x3^2

In [31]:
nv = 3
g = Graph{Undirected}(nv)
add_edge!(g, 1, 3)
add_edge!(g, 1, 2)


R, x = PolynomialRing(QQ, ["x$i" for i in 1:nv]) # Definimos el anillo de polinomios con n = #vercies del grafo variables
G = polisGrafo(g, 3, R)
println(length(G))

5


In [32]:
nulla(G,56)

CERTv
CERT
Creados
The system of equations F is infeasible


11-element Vector{Float64}:
 -0.2833333333333332
 -0.2611111111111111
 -0.26111111111111096
 -3.110349676662262e-18
 -0.02777777777777775
  0.06666666666666658
  0.04444444444444461
  9.331049029986785e-18
  0.06666666666666667
 -0.027777777777777842
  0.04444444444444448